In [ ]:
build = 'archive' # 'archive' uses the neurolibre archive of the data., 'latest' would download the latest versions of the data.
notebook = 'figures'

<head>

<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=STIX+Two+Text:ital,wght@0,400;0,700;1,400&display=swap" rel="stylesheet">
</head>


<p>
<center>
<b>
<h3>
Analysis code for the paper "RF shimming in the cervical spinal cord at 7T"
</h3>
</b>
</center>


# 1 &nbsp; &nbsp; | &nbsp; &nbsp; Data

The data can be downloaded from: https://openneuro.org/datasets/ds004906

The structure of the input dataset is as follows (JSON sidecars are not listed for clarity):

```{toggle}
~~~
ds004906
├── CHANGES
├── README
├── dataset_description.json
├── participants.json
├── participants.tsv
├── sub-01
│   ├── anat
│   │   ├── sub-01_acq-CP_T1w.nii.gz
│   │   ├── sub-01_acq-CP_T2starw.nii.gz
│   │   ├── sub-01_acq-CoV_T1w.nii.gz
│   │   ├── sub-01_acq-CoV_T2starw.nii.gz
│   │   ├── sub-01_acq-SAReff_T2starw.nii.gz
│   │   ├── sub-01_acq-patient_T2starw.nii.gz
│   │   ├── sub-01_acq-phase_T2starw.nii.gz
│   │   ├── sub-01_acq-target_T2starw.nii.gz
│   │   ├── sub-01_acq-volume_T2starw.nii.gz
│   └── fmap
│       ├── sub-01_acq-anatCP_TB1TFL.nii.gz
│       ├── sub-01_acq-anatCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-anatSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-anatpatient_TB1TFL.nii.gz
│       ├── sub-01_acq-anatphase_TB1TFL.nii.gz
│       ├── sub-01_acq-anattarget_TB1TFL.nii.gz
│       ├── sub-01_acq-anatvolume_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCP_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-fampSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-famppatient_TB1TFL.nii.gz
│       ├── sub-01_acq-fampphase_TB1TFL.nii.gz
│       ├── sub-01_acq-famptarget_TB1TFL.nii.gz
│       └── sub-01_acq-fampvolume_TB1TFL.nii.gz
├── sub-02
├── sub-03
├── sub-04
└── sub-05
~~~
```

Test citation: {cite:p}`DAstous2023`.

# 2 &nbsp; &nbsp; | &nbsp; &nbsp; Overview of processing pipeline

For each subject:

- Process anat/T2starw (GRE)
  - Segment the spinal cord (SC)
  - Label vertebral levels using existing manual disc labels
  - Create a mask of the cerebrospinal fluid (CSF)
  - Extract the SC/CSF magnitude signal to assess the stability of the flip angle across shim methods
- Process fmap/TFL (flip angle maps)
  - Register each B1 map (CP, CoV, etc.) to the GRE scan
  - Apply the computed warping field to bring the segmentation and vertebral levels to the B1 map
  - Convert the B1 map to nT/V units
  - Extract the B1 map value within the SC

>Slow processes are indicated with the emoji ⏳

# 3 &nbsp; &nbsp; | &nbsp; &nbsp; Requirements

* Install [Spinal Cord Toolbox](https://spinalcordtoolbox.com/user_section/installation.html), eg

```shell
# Install Python libaries
!wget -O requirements.txt https://raw.githubusercontent.com/shimming-toolbox/rf-shimming-7t/main/requirements.txt
!pip install -r requirements.txt

# Install SCT ⏳
!git clone --depth 1 https://github.com/spinalcordtoolbox/spinalcordtoolbox.git
!yes | spinalcordtoolbox/install_sct
os.environ['PATH'] += f":/content/spinalcordtoolbox/bin"
```

* Download the project repository

```shell
git clone https://github.com/shimming-toolbox/rf-shimming-7t-neurolibre.git
cd rf-shimming-7t-neurolibre
```

* Install requirements

```shell
pip install -e binder/requirements.txt
```

* Download data

```shell
cd content
repo2data -r ../binder/data_requirement.json
```


# 3 &nbsp; &nbsp; | &nbsp; &nbsp; Environment setup

In a Python shell:

Import the necessary modules.

In [ ]:
from datetime import datetime, timedelta
import os
import re
import json
import subprocess
import glob
import itertools
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import pandas as pd
from scipy.interpolate import interp1d
from scipy.ndimage import uniform_filter1d
from scipy.stats import f_oneway, ttest_rel
import shutil
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.anova import AnovaRM
import statsmodels.api as sm

Define variables.

In [ ]:
os.chdir("../data/rf-shimming-7t/ds004906")

path_data = os.getcwd()
print(f"path_data: {path_data}")
path_labels = os.path.join(path_data, "derivatives", "labels")
path_qc = os.path.join(path_data, "qc")
shim_modes = ["CP", "patient", "volume", "phase", "CoV", "target", "SAReff"]
shim_modes_MPRAGE = ["CP", "CoV"] 

print(f"shim_modes: {shim_modes}")
subjects = sorted(glob.glob("sub-*"))
print(f"subjects: {subjects}")

# Create output folder
path_results = os.path.join(path_data, 'derivatives', 'results')
os.makedirs(path_results, exist_ok=True)

# 3 &nbsp; &nbsp; | &nbsp; &nbsp; Overview of processing pipeline


Run segmentation on GRE scan


In [ ]:

if notebook != 'figures':
    for subject in subjects:
        os.chdir(os.path.join(path_data, subject, "anat"))
        # The "CoV" RF shimming scenario was chosen as the segmentation baseline due to the more homogeneous signal intensity in the I-->S direction, which results in a better segmentation peformance in the C7-T2 region
        fname_manual_seg = os.path.join(path_labels, subject, "anat", f"{subject}_acq-CoV_T2starw_label-SC_seg.nii.gz")
        if os.path.exists(fname_manual_seg):
            # Manual segmentation already exists. Copy it to local folder
            print(f"{subject}: Manual segmentation found\n")
            shutil.copyfile(fname_manual_seg, f"{subject}_acq-CoV_T2starw_seg.nii.gz")
            # Generate QC report to make sure the manual segmentation is correct
            !sct_qc -i {subject}_acq-CoV_T2starw.nii.gz -s {subject}_acq-CoV_T2starw_seg.nii.gz -p sct_deepseg_sc -qc {path_qc} -qc-subject {subject}
        else:
            # Manual segmentation does not exist. Run automatic segmentation.
            print(f"{subject}: Manual segmentation not found")
            !sct_deepseg_sc -i {subject}_acq-CoV_T2starw.nii.gz -c t2s -qc {path_qc}

Copy CSF masks from the derivatives folder.

For more details about how these masks were created, see: [https://github.com/shimming-toolbox/rf-shimming-7t/issues/67](https://github.com/shimming-toolbox/rf-shimming-7t/issues/67).

In [ ]:
if notebook != 'figures':
    for subject in subjects:
        os.chdir(os.path.join(path_data, subject, "anat"))
        fname_manual_seg = os.path.join(path_labels, subject, "anat", f"{subject}_acq-CoV_T2starw_label-CSF_seg.nii.gz")
        if os.path.exists(fname_manual_seg):
            # Manual segmentation already exists. Copy it to local folder
            print(f"{subject}: Manual segmentation found\n")
            shutil.copyfile(fname_manual_seg, f"{subject}_acq-CoV_T2starw_label-CSF_seg.nii.gz")
            # Generate QC report to make sure the manual segmentation is correct
            !sct_qc -i {subject}_acq-CoV_T2starw.nii.gz -s {subject}_acq-CoV_T2starw_label-CSF_seg.nii.gz -p sct_deepseg_sc -qc {path_qc} -qc-subject {subject}
        else:
            # Manual segmentation does not exist. Panic!
            print(f"{subject}: Manual segmentation not found 😱")

Crop GRE scan for faster processing and better registration results


In [ ]:
if notebook != 'figures':
    for subject in subjects:
        os.chdir(os.path.join(path_data, subject, "anat"))
        !sct_crop_image -i {subject}_acq-CoV_T2starw.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop.nii.gz
        !sct_crop_image -i {subject}_acq-CoV_T2starw_seg.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop_seg.nii.gz
        !sct_crop_image -i {subject}_acq-CoV_T2starw_label-CSF_seg.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop_label-CSF_seg.nii.gz

Label vertebrae on GRE scan

In [ ]:

# Given the low resolution of the GRE scan, the automatic detection of C2-C3 disc is unreliable. Therefore we need to use the manual disc labels that are part of the dataset.
if notebook != 'figures':
    for subject in subjects:
        os.chdir(os.path.join(path_data, subject, "anat"))
        fname_label_discs = os.path.join(path_data, "derivatives", "labels", subject, "anat", f"{subject}_acq-CoV_T2starw_label-discs_dseg.nii.gz")
        !sct_label_utils -i {subject}_acq-CoV_T2starw_crop_seg.nii.gz -disc {fname_label_discs} -o {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz
        # Generate QC report to assess labeled segmentation
        !sct_qc -i {subject}_acq-CoV_T2starw_crop.nii.gz -s {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -p sct_label_vertebrae -qc {path_qc} -qc-subject {subject}

Register *_T2starw to CoV_T2starw ⏳

In [ ]:
if notebook != 'figures':
    for subject in subjects:
        os.chdir(os.path.join(path_data, subject, "anat"))
        for shim_mode in shim_modes:
            # Don't do it for CoV_T2starw
            if shim_mode != 'CoV':
                !sct_register_multimodal -i {subject}_acq-{shim_mode}_T2starw.nii.gz -d {subject}_acq-CoV_T2starw_crop.nii.gz -dseg {subject}_acq-CoV_T2starw_crop_seg.nii.gz -param step=1,type=im,algo=dl -qc {path_qc}

Extract the signal intensity on the GRE scan within the spinal cord between levels C3 and T2 (included), which correspond to the region where RF shimming was prescribed.

In [ ]:

if notebook != 'figures':
    for subject in subjects:
        os.chdir(os.path.join(path_data, subject, "anat"))
        for shim_mode in shim_modes:
            # Shim methods are registered to the CoV T2starw scan, so we need to use the added suffix to identify them
            if shim_mode == 'CoV':
                file_suffix = 'crop'
            else:
                file_suffix = 'reg'
            fname_result_sc = os.path.join(path_results, f"{subject}_acq-{shim_mode}_T2starw_label-SC.csv")
            !sct_extract_metric -i {subject}_acq-{shim_mode}_T2starw_{file_suffix}.nii.gz -f {subject}_acq-CoV_T2starw_crop_seg.nii.gz -method wa -vert 3:9 -vertfile {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -perslice 1 -o {fname_result_sc}
            fname_result_csf = os.path.join(path_results, f"{subject}_acq-{shim_mode}_T2starw_label-CSF.csv")
            !sct_extract_metric -i {subject}_acq-{shim_mode}_T2starw_{file_suffix}.nii.gz -f {subject}_acq-CoV_T2starw_crop_label-CSF_seg.nii.gz -method wa -vert 3:9 -vertfile {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -perslice 1 -o {fname_result_csf}

Make figure of CSF/SC signal ratio from T2starw scan

In [ ]:
# Make figure of CSF/SC signal ratio from T2starw scan

# Go back to root data folder
os.chdir(path_data)

def smooth_data(data, window_size=50):
    """ Apply a simple moving average to smooth the data. """
    return uniform_filter1d(data, size=window_size, mode='nearest')

# Fixed grid for x-axis
x_grid = np.linspace(0, 1, 100)

# z-slices corresponding to levels C3 to T2 on the PAM50 template. These will be used to scale the x-label of each subject.
original_vector = np.array([907, 870, 833, 800, 769, 735, 692, 646])

# Normalize the PAM50 z-slice numbers to the 1-0 range (to show inferior-superior instead of superior-inferior)
min_val = original_vector.min()
max_val = original_vector.max()
normalized_vector = 1 - ((original_vector - min_val) / (max_val - min_val))

# Use this normalized vector as x-ticks
custom_xticks = normalized_vector

# Vertebral level labels
vertebral_levels = ["C3", "C4", "C5", "C6", "C7", "T1", "T2"]
label_positions = normalized_vector[:-1] + np.diff(normalized_vector) / 2

# Number of subjects determines the number of rows in the subplot
n_rows = len(subjects)

# Create a figure with multiple subplots
fig, axes = plt.subplots(n_rows, 1, figsize=(10, 6 * n_rows))
font_size = 18
line_width = 3

# Check if axes is an array or a single object
if n_rows == 1:
    axes = [axes]

# Data storage for statistics
data_stats = []

# Iterate over each subject and create a subplot
for i, subject in enumerate(subjects):
    ax = axes[i]

    for shim_mode in shim_modes:
        # Initialize list to collect data for this shim method
        method_data = []

        # Get signal in SC
        file_csv = os.path.join(path_results, f"{subject}_acq-{shim_mode}_T2starw_label-SC.csv")
        df = pd.read_csv(file_csv)
        data_sc = df['WA()']
        data_sc_smoothed = smooth_data(data_sc)

        # Get signal in CSF
        file_csv = os.path.join(path_results, f"{subject}_acq-{shim_mode}_T2starw_label-CSF.csv")
        df = pd.read_csv(file_csv)
        data_csf = df['WA()']
        data_csf_smoothed = smooth_data(data_csf)
        
        # Compute ratio
        data_sc_csf_ratio = data_csf_smoothed / data_sc_smoothed

        # Normalize the x-axis to a 1-0 scale for each subject (to go from superior-inferior direction)
        x_subject = np.linspace(1, 0, len(data_sc_csf_ratio))

        # Interpolate to the fixed grid
        interp_func = interp1d(x_subject, data_sc_csf_ratio, kind='linear', bounds_error=False, fill_value='extrapolate')
        resampled_data = interp_func(x_grid)

        method_data.append(resampled_data)

        # If there's data for this shim method, plot it and compute stats
        if method_data:
            # Plotting each file's data separately
            for resampled_data in method_data:
                ax.plot(x_grid, resampled_data, label=f"{shim_mode}", linewidth=line_width)
        
            # Compute stats on the non-resampled data (to avoid interpolation errors)
            mean_data = np.mean(data_sc_csf_ratio)
            sd_data = np.std(data_sc_csf_ratio)
            data_stats.append([subject, shim_mode, mean_data, sd_data])

    # Set x-ticks and labels for the bottom subplot
    if i == n_rows - 1:  # Check if it's the last subplot
        # Create a secondary axis for vertebral level labels
        secax = ax.secondary_xaxis('bottom')
        secax.set_xticks(label_positions)
        secax.set_xticklabels(vertebral_levels, fontsize=font_size-4)
        secax.tick_params(axis='x', which='major', length=0)  # Hide tick marks

    ax.set_xticks(custom_xticks)
    ax.set_xticklabels([''] * len(custom_xticks))
    ax.tick_params(axis='x', which='major', length=0)  # Hide tick marks for the primary axis

    ax.set_title(f'{subject}', fontsize=font_size)
    ax.set_ylabel('CSF/Cord T2starw signal ratio', fontsize=font_size)
    ax.tick_params(axis='y', which='major', labelsize=font_size-4)

    # Add legend only to the first subplot
    if i == 0:
        ax.legend(fontsize=font_size)

    ax.grid(True)

# Adjust the layout so labels and titles do not overlap
plt.tight_layout()
plt.savefig(os.path.join(path_results, 'fig_gre_csf-sc_ratio.png'), dpi=300, format='png')
plt.show()

Perform statistics

In [ ]:
# Perform statistics

# Convert to DataFrame and save to CSV
df_stats = pd.DataFrame(data_stats, columns=['Subject', 'Shim_Mode', 'Average', 'Standard_Deviation'])
df_stats.to_csv(os.path.join(path_results, 'stats_t2starw.csv'), index=False)

# Compute statistics across subjects
grouped_means = df_stats.groupby('Shim_Mode').agg({'Average': ['mean', 'std'], 'Standard_Deviation': ['mean', 'std']})

# Format mean ± standard deviation
grouped_means['Average_formatted'] = grouped_means['Average']['mean'].map("{:.2f}".format) + " ± " + grouped_means['Average']['std'].map("{:.2f}".format)
grouped_means['Standard_Deviation_formatted'] = grouped_means['Standard_Deviation']['mean'].map("{:.2f}".format) + " ± " + grouped_means['Standard_Deviation']['std'].map("{:.2f}".format)

# Drop multi-level index and only keep formatted columns
grouped_means = grouped_means.drop(columns=['Average', 'Standard_Deviation'])
grouped_means.columns = ['Average', 'Standard_Deviation']  # Rename columns for clarity
grouped_means.reset_index().to_csv(os.path.join(path_results, 'stats_t2starw_average_across_subjects.csv'), index=False)

# Perform Repeated Measures ANOVA
aovrm = AnovaRM(df_stats, 'Average', 'Subject', within=['Shim_Mode'])
res = aovrm.fit()
print(res.summary())

# Perform Post Hoc paired t-tests

# Filter out subjects that don't have all shim modes
shim_modes = df_stats['Shim_Mode'].unique()
valid_subjects = df_stats.groupby('Subject').filter(lambda x: all(mode in x['Shim_Mode'].values for mode in shim_modes))

# Pairwise comparisons
pairs = list(itertools.combinations(shim_modes, 2))
p_values = []

for pair in pairs:
    data1 = valid_subjects[valid_subjects['Shim_Mode'] == pair[0]]['Average']
    data2 = valid_subjects[valid_subjects['Shim_Mode'] == pair[1]]['Average']
    _, p_value = ttest_rel(data1, data2)
    p_values.append(p_value)

# Function for Benjamini-Hochberg FDR correction
def benjamini_hochberg(p_values):
    n = len(p_values)
    sorted_p_values = np.sort(p_values)
    sorted_index = np.argsort(p_values)
    adjusted_p_values = np.zeros(n)

    for i, p in enumerate(sorted_p_values):
        adjusted_p_values[sorted_index[i]] = min(p * n / (i + 1), 1)

    return adjusted_p_values

# Applying Benjamini-Hochberg FDR correction
adjusted_p_values = benjamini_hochberg(p_values)

# Creating a summary DataFrame
comparison_results = pd.DataFrame({'Pair': pairs, 'P-Value': p_values, 'Adjusted P-Value': adjusted_p_values})

# Save to CSV
comparison_results.to_csv(os.path.join(path_results, 'stats_t2starw_paired_ttest.csv'), index=False)

print(f"Paired t-tests:\n{comparison_results}")

# References 

```{bibliography}
:filter: docname in docnames
```